In [20]:
import os
import logging
from pathlib import Path
from PIL import Image
import torch
from transformers import pipeline
import joblib
import numpy as np

from torchvision import transforms
from torch import nn
import torch.nn.functional as F

# Disable TF and set up logging
os.environ["USE_TF"] = "0"
logging.basicConfig(level=logging.INFO, format="[%(levelname)s] %(message)s")

PROJECT_ROOT = Path.cwd().resolve().parents[0] if Path.cwd().name == "notebooks" else Path.cwd()
DATA_ROOT = PROJECT_ROOT / "data"
MODELS_ROOT = PROJECT_ROOT / "models"
print("Pipeline initialized ✅")


Pipeline initialized ✅


In [21]:
class OCRModel(nn.Module):
    def __init__(self, num_classes=46):
        super(OCRModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.conv3 = nn.Conv2d(64, 128, 3, 1)
        self.fc1 = nn.Linear(128 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, num_classes)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

ocr_model_path = MODELS_ROOT / "devanagari_cnn_v1.pth"
ocr_model = OCRModel(num_classes=46)  # adjust num_classes if your dataset differs

# Safe model loading
if ocr_model_path.exists():
    checkpoint = torch.load(ocr_model_path, map_location="cpu")
    ocr_model.load_state_dict(checkpoint["model_state_dict"])
    class_names = checkpoint.get("class_names", [])
    ocr_model.eval()
    logging.info(f"OCR model loaded ✅  ({ocr_model_path.name})")
    if class_names:
        logging.info(f"Loaded {len(class_names)} character classes.")
else:
    logging.warning(f"OCR model not found at {ocr_model_path}, using untrained model 🚧")
    class_names = []


[INFO] OCR model loaded ✅  (devanagari_cnn_v1.pth)
[INFO] Loaded 46 character classes.


In [22]:
from PIL import Image
from torchvision import transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def preprocess_image(image_path):
    """
    Load an image, convert to grayscale, resize to 64x64 (same as training),
    normalize, and add batch dimension.
    """
    image = Image.open(image_path).convert("L")  # grayscale

    transform = transforms.Compose([
        transforms.Resize((64, 64)),              # MUST match training size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5]),
    ])

    tensor = transform(image).unsqueeze(0)       # shape: [1, 1, 64, 64]
    # Optional debug print:
    # print("DEBUG tensor shape:", tensor.shape)

    return tensor.to(device)


def ocr_predict(image_tensor):
    """Predict character from OCR model."""
    with torch.no_grad():
        logits = ocr_model(image_tensor)
        pred = torch.argmax(logits, dim=1).item()
    return pred

def transliterate_character(pred_index, mapping_dict):
    """Convert model output index → phonetic representation."""
    return mapping_dict.get(pred_index, "?")

def reconstruct_text(sequence, bigram_probs):
    """Use probabilistic smoothing to predict missing pieces."""
    text = "".join(sequence)
    for word, prob in bigram_probs.items():
        if word in text:
            text = text.replace(word, word)  # placeholder for actual logic
    return text

def detect_language(text):
    X = vectorizer.transform([text])
    probs = lang_model.predict_proba(X)[0]
    classes = lang_model.classes_
    return classes[np.argmax(probs)], np.max(probs)

def translate_text(text, lang):
    result = translator(text, clean_up_tokenization_spaces=True)
    return result[0]["translation_text"]


In [23]:
import pandas as pd
mapping_file = DATA_ROOT / "phonetic_mappings" / "all_script_phonetic_data.csv"

# Load the CSV
mapping_df = pd.read_csv(mapping_file)

# Confirm structure
print("Columns:", mapping_df.columns.tolist())

# Build dictionary safely using attribute access
if "ITRANS" in mapping_df.columns:
    mapping_dict = {i: getattr(row, "ITRANS") for i, row in enumerate(mapping_df.itertuples())}
else:
    # If the column name differs slightly, like "itrans" or "Itrans", normalize
    itrans_col = [c for c in mapping_df.columns if c.lower() == "itrans"][0]
    mapping_dict = {i: getattr(row, itrans_col) for i, row in enumerate(mapping_df.itertuples())}

logging.info(f"Loaded {len(mapping_dict)} transliteration mappings ✅")


[INFO] Loaded 112 transliteration mappings ✅


Columns: ['Unicode', 'Relative Offset', 'Devanagari', 'ITRANS', 'Notes', 'Valid Vector Representation', 'is_vowel', 'is_consonant', 'nukta', 'halanta', 'anusvara', 'misc', 'short_vowel', 'long_vowel', 'weak', 'medium', 'strong', 'independent_vowel', 'dependent_vowel', 'plosive', 'fricative', 'Central-approximant', 'Lateral-approximant', 'flap', 'velar', 'palatal', 'retroflex', 'dental', 'labial', 'aspirated', 'not_aspirated', 'voiced', 'unvoiced', 'nasal', 'not_nasal', 'front', 'central', 'back', 'close', 'close-mid', 'open-mid', 'open', 'rounded', 'not_rounded']


In [24]:
class AncientScriptAI:
    def __init__(self):
        self.ocr_model = ocr_model
        self.mapping = mapping_dict
        self.lang_clf = lang_model
        self.vectorizer = vectorizer
        self.translator = translator

    def process_image(self, image_path):
        logging.info(f"Processing {image_path.name}")
        tensor = preprocess_image(image_path)
        pred_idx = ocr_predict(tensor)
        translit = transliterate_character(pred_idx, self.mapping)
        reconstructed = reconstruct_text([translit], {})
        lang, conf = detect_language(reconstructed)
        translation = translate_text(reconstructed, lang)
        return {
            "image": str(image_path),
            "recognized_text": translit,
            "reconstructed_text": reconstructed,
            "language": lang,
            "confidence": round(conf, 3),
            "translation": translation
        }


In [25]:
from pathlib import Path

# Set up your base directories
PROJECT_ROOT = Path.cwd().resolve().parents[0] if Path.cwd().name == "notebooks" else Path.cwd()
MODELS_ROOT = PROJECT_ROOT / "models"
MODELS_ROOT.mkdir(exist_ok=True)

print("Models directory set to:", MODELS_ROOT)


Models directory set to: /Users/jyotirmoy/Desktop/Image/ancient-script-ai/models


In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import joblib
import pandas as pd

# Small starter dataset (you can expand it)
data = {
    "text": [
        "कर्मणा जायते पुरुषः", "ज्ञानं योगेन साध्यते",   # Sanskrit
        "मेरा घर बड़ा है", "वह बाज़ार गया था",           # Hindi
        "माझे घर मोठे आहे", "तो बाजारात गेला होता"        # Marathi
    ],
    "language": ["sanskrit", "sanskrit", "hindi", "hindi", "marathi", "marathi"]
}

lang_df = pd.DataFrame(data)

# Train Naive Bayes classifier
vectorizer = CountVectorizer(analyzer="char", ngram_range=(1,3))
X = vectorizer.fit_transform(lang_df["text"])
y = lang_df["language"]

lang_model = MultinomialNB()
lang_model.fit(X, y)

# Save models freshly compiled in this environment
MODELS_ROOT.mkdir(exist_ok=True)
joblib.dump(lang_model, MODELS_ROOT / "language_id_model.pkl")
joblib.dump(vectorizer, MODELS_ROOT / "language_vectorizer.pkl")

print("✅ Language model retrained and saved successfully!")


✅ Language model retrained and saved successfully!


In [27]:
from transformers import pipeline
import os

# Ensure TensorFlow stays disabled
os.environ["USE_TF"] = "0"

# Load the multilingual MarianMT model for translation
translator = pipeline(
    "translation",
    model="Helsinki-NLP/opus-mt-mul-en",
    framework="pt"
)

print("Translation pipeline loaded ✅")


Device set to use mps:0


Translation pipeline loaded ✅


In [28]:

import joblib

# Paths to your trained models
lang_model_path = MODELS_ROOT / "language_id_model.pkl"
vectorizer_path = MODELS_ROOT / "language_vectorizer.pkl"

# Load models
lang_model = joblib.load(lang_model_path)
vectorizer = joblib.load(vectorizer_path)

print("Language ID model and vectorizer loaded ✅")

pipeline_ai = AncientScriptAI()

sample_image = DATA_ROOT / "samples" / "test_devanagari_char.png"
result = pipeline_ai.process_image(sample_image)

print("\n📜 Final Result:")
for k, v in result.items():
    print(f"{k:20}: {v}")


[INFO] Processing test_devanagari_char.png


Language ID model and vectorizer loaded ✅


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x107648 and 8192x256)

In [29]:
import torch
from torch import nn
import torch.nn.functional as F

class ScriptCNN(nn.Module):
    """
    CNN architecture MUST match the one used during training.
    64x64 → conv/pool → 32x32 → 16x16 → 8x8 → flatten 8192.
    """
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)  # halves H and W

        # 64x64 → pool → 32x32 → pool → 16x16 → pool → 8x8
        self.fc1 = nn.Linear(128 * 8 * 8, 256)  # 8192 inputs
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)  # flatten: should be (batch, 8192)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [30]:
from pathlib import Path

MODELS_ROOT = PROJECT_ROOT / "models"
ckpt_path = MODELS_ROOT / "devanagari_cnn_v1.pth"  # <-- change if your file name differs

checkpoint = torch.load(ckpt_path, map_location="cpu")
class_names = checkpoint["class_names"]

ocr_model = ScriptCNN(num_classes=len(class_names))
ocr_model.load_state_dict(checkpoint["model_state_dict"])
ocr_model.eval()

print("Loaded OCR model ✅")
print("Num classes:", len(class_names))


Loaded OCR model ✅
Num classes: 46


In [31]:
from torchvision import transforms
from PIL import Image

IMG_SIZE = 64

inference_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),      # NO normalization if you didn't use it in training
])


device = torch.device("mps" if torch.mps.is_available() else "cpu")

def preprocess_image(image_path):
    """
    Load an image, convert to grayscale, resize to 64x64 (same as training),
    normalize, and add batch dimension.
    """
    image = Image.open(image_path).convert("L")  # grayscale

    transform = transforms.Compose([
        transforms.Resize((64, 64)),              # MUST match training size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5]),
    ])

    tensor = transform(image).unsqueeze(0)       # shape: [1, 1, 64, 64]
    # Optional debug print:
    # print("DEBUG tensor shape:", tensor.shape)

    return tensor.to(device)



In [32]:
t = preprocess_image(sample_image)
print(t.shape)   # should be: torch.Size([1, 1, 64, 64])


torch.Size([1, 1, 64, 64])


In [33]:
def ocr_predict(image_tensor: torch.Tensor) -> int:
    """Return predicted class index for a single image tensor."""
    with torch.no_grad():
        logits = ocr_model(image_tensor)
        pred_idx = torch.argmax(logits, dim=1).item()
    return pred_idx


In [34]:
def ocr_predict_label(image_tensor: torch.Tensor) -> str:
    idx = ocr_predict(image_tensor)
    return class_names[idx]


In [35]:
class AncientScriptAI:
    def __init__(self):
        self.ocr_model = ocr_model
        self.class_names = class_names
        self.mapping = mapping_dict              # from your transliteration CSV
        self.lang_clf = lang_model              # joblib loaded
        self.vectorizer = vectorizer
        self.translator = translator

    def process_image(self, image_path):
        logging.info(f"Processing {image_path.name}")
        
        tensor = preprocess_image(image_path)
        pred_idx = ocr_predict(tensor)
        class_label = self.class_names[pred_idx]

        # OPTIONAL: map class_label → actual Devanagari char if you have such a dict
        # For now we’ll assume transliteration mapping is by character index:
        translit = transliterate_character(pred_idx, self.mapping)

        reconstructed = reconstruct_text([translit], {})  # simple for now
        lang, conf = detect_language(reconstructed)
        translation = translate_text(reconstructed, lang)

        return {
            "image": str(image_path),
            "predicted_class": class_label,
            "recognized_text": translit,
            "reconstructed_text": reconstructed,
            "language": lang,
            "confidence": round(conf, 3),
            "translation": translation
        }


In [36]:
# Assuming you already loaded your checkpoint
# and have `class_names` like in 01 notebook:

print("Number of classes:", len(class_names))
for i, name in enumerate(class_names):
    print(i, name)


Number of classes: 46
0 character_10_yna
1 character_11_taa
2 character_12_thaa
3 character_13_daa
4 character_14_dhaa
5 character_15_adna
6 character_16_tabala
7 character_17_tha
8 character_18_da
9 character_19_dha
10 character_1_ka
11 character_20_na
12 character_21_pa
13 character_22_pha
14 character_23_ba
15 character_24_bha
16 character_25_ma
17 character_26_yaw
18 character_27_ra
19 character_28_la
20 character_29_waw
21 character_2_kha
22 character_30_motosaw
23 character_31_petchiryakha
24 character_32_patalosaw
25 character_33_ha
26 character_34_chhya
27 character_35_tra
28 character_36_gya
29 character_3_ga
30 character_4_gha
31 character_5_kna
32 character_6_cha
33 character_7_chha
34 character_8_ja
35 character_9_jha
36 digit_0
37 digit_1
38 digit_2
39 digit_3
40 digit_4
41 digit_5
42 digit_6
43 digit_7
44 digit_8
45 digit_9


In [37]:

class_to_dev = {
    "character_1_ka":  "क",
    "character_2_kha": "ख",
    "character_3_ga":  "ग",
    "character_4_gha": "घ",
    "character_5_kna": "ङ",
    "character_6_cha":  "च",
    "character_7_chha": "छ",
    "character_8_ja":  "ज",
    "character_9_jha": "झ",
    "character_10_yna": "ञ",
    "character_11_taamatar": "ट",
    "character_12_thaa": "ठ",
    "character_13_daa": "ड",
    "character_14_dhaa": "ढ",
    "character_15_adna": "ण",
    "character_16_tabala": "त",
    "character_17_tha": "थ",
    "character_18_da": "द",
    "character_19_dha": "ध",
    "character_20_na": "न",
    "character_21_pa": "प",
    "character_22_pha": "फ",
    "character_23_ba": "ब",
    "character_24_bha": "भ",
    "character_25_ma": "म",
    "character_26_yaw": "य",
    "character_27_ra": "र",
    "character_28_la": "ल",
    "character_29_waw": "व",
    "character_30_motosaw": "श",
    "character_31_petchiryakha": "ष",
    "character_32_patalosaw": "स",
    "character_33_ha": "ह",
    "character_34_chhya": "क्ष",
    "character_35_tra": "त्र",
    "character_36_gya": "ज्ञ",
    "digit_0": "०",
    "digit_1": "१", 
    "digit_2": "२",
    "digit_3": "३",
    "digit_4": "४",
    "digit_5": "५",
    "digit_6": "६",
    "digit_7": "७",
    "digit_8": "८",
    "digit_9": "९"

    
}

missing = [c for c in class_names if c not in class_to_dev]
print("Still to fill:", missing)


Still to fill: ['character_11_taa']


In [39]:
import pandas as pd
from pathlib import Path

phonetic_path = DATA_ROOT / "phonetic_mappings"/ "all_script_phonetic_data.csv"
phonetic_df = pd.read_csv(phonetic_path)

# keep only rows that have both Devanagari and ITRANS
phonetic_df = phonetic_df.dropna(subset=["Devanagari", "ITRANS"])

dev_to_itrans = dict(zip(phonetic_df["Devanagari"], phonetic_df["ITRANS"]))

print("Total mapping entries:", len(dev_to_itrans))
print("Example:", list(dev_to_itrans.items())[:10])


Total mapping entries: 111
Example: [('ऀ', 'ऀ'), ('ँ', '.n'), ('ं', '.n'), ('ः', 'H'), ('ऄ', 'ऄ'), ('अ', 'a'), ('आ', 'A'), ('इ', 'i'), ('ई', 'I'), ('उ', 'u')]


In [40]:
def class_to_translit(class_label: str):
    """
    Convert a CNN class label like 'character_1_ka'
    into (devanagari_char, itrans_string).
    """
    dev_char = class_to_dev.get(class_label, "?")
    itrans = dev_to_itrans.get(dev_char, dev_char)
    return dev_char, itrans


In [33]:
print(class_to_translit("character_1_ka"))   # should be ('क', 'ka')
print(class_to_translit("character_23_ba"))  # should be ('ब', 'ba') after you fill mapping


('क', 'ka')
('ब', 'ba')


In [41]:
class AncientScriptAI:
    def __init__(self):
        self.ocr_model = ocr_model
        self.class_names = class_names
        self.class_to_dev = class_to_dev
        self.dev_to_itrans = dev_to_itrans

        # you still have lang_clf, vectorizer, translator
        # but we won't use them for single chars

    def process_image(self, image_path):
        logging.info(f"Processing {image_path.name}")

        tensor = preprocess_image(image_path)  # 64x64 → tensor
        pred_idx = ocr_predict(tensor)
        class_label = self.class_names[pred_idx]

        dev_char, itrans = class_to_translit(class_label)

        # For a single character, STOP here.
        # Don't run language ID or translation yet.
        return {
            "image": str(image_path),
            "predicted_class": class_label,
            "recognized_devanagari": dev_char,
            "recognized_itrans": itrans,
        }


In [35]:
pipeline_ai = AncientScriptAI()

sample_image = DATA_ROOT / "samples" / "test_devanagari_char.png"
result = pipeline_ai.process_image(sample_image)

print("\n📜 Final Result:")
for k, v in result.items():
    print(f"{k:20}: {v}")


[INFO] Processing test_devanagari_char.png



📜 Final Result:
image               : /Users/jyotirmoy/Desktop/Image/ancient-script-ai/data/samples/test_devanagari_char.png
predicted_class     : character_1_ka
recognized_devanagari: क
recognized_itrans   : ka


In [42]:
import json, datetime
outputs_dir = PROJECT_ROOT / "outputs"
outputs_dir.mkdir(exist_ok=True)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = outputs_dir / f"result_{timestamp}.json"

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

logging.info(f"Saved output to {output_file}")


NameError: name 'result' is not defined